In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew

In [3]:
from crewai import Agent, Task, Crew

In [6]:
import os
from google.colab import userdata
# Check if OPENAI_API_KEY environment variable is set and retrieve it, if not, return an empty string
api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_MODEL_NAME"] = 'chatgpt-4o-latest'

##Creating Agent
  In this MAS we would have three agents, namely --
  1. Planner          :   is the content planner for our blog posts
  2. Content Writer   :   this is the agent that actually writes the code for us
  3. Editor           :   this is the agent that would be responsible for editing and proof checking our blog post.

In [7]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article about the topic: {topic}."
              "You collect information that helps the audience learn something and make informed decisions. "
              "Your work is the basis for the Content Writer to write an article on this topic.",
    allow_delegation=False,
	  verbose=True
)

In [8]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",
    backstory="You're working on a writing a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic. "
              "You follow the main objectives and direction of the outline, as provide by the Content Planner. "
              "You also provide objective and impartial insights and back them up with information provide by the Content Planner. "
              "You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [9]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating the Tasks
In here we define the tasks that our agents would be performing, each agent could have. multiple tasks, or a single task (generally a single task is prefered). In here we have three agents each with a single task :
  1. Planner has the task of planning
  2. writer has the task of writing
  3. editor has the task of editing

This seems very obvious, but how crew works is by dividing the tasks into multiple small sections, which a specialized agent could work on.

In [10]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [11]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		    "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [12]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

In [16]:
crew = Crew(
    agents = [planner, writer, editor],
    tasks = [plan, write, edit],
    verbose = 2
)

In [20]:
result = crew.kickoff(inputs={"topic": "Quantum Chips and Computing"})



> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: Quantum Chips and Computing

Outline:
I. Introduction
    A. Brief overview of quantum computing
    B. Importance of quantum chips in quantum computing
II. Latest Trends in Quantum Chips and Computing
    A. Advancements in quantum chip technology
    B. Collaboration between key players in the industry
III. Key Players in Quantum Chips and Computing
    A. IBM
    B. Google
    C. Intel
IV. Noteworthy News in Quantum Chips and Computing
    A. Breakthroughs in quantum chip development
    B. Impact of quantum computing on various industries
V. Target Audience Analysis
    A. Tech enthusiasts interested in cutting-edge technology
    B. IT professionals looking to stay updated on quantum computing
    C. Business leaders exploring the potential applications of quantum computing
VI. SEO Keywords
    A. Quantum chips
    B. Quantum computing
    C. Quantum technology
VII. Call to Act

In [21]:
from IPython.display import Markdown
Markdown(result)

# Quantum Chips and Computing: Unleashing the Power of Quantum Technology

## Introduction

Quantum computing is a groundbreaking field that is reshaping the landscape of technology by offering solutions to complex problems that traditional computers struggle with. At the core of quantum computing are quantum chips, vital components that facilitate the processing of quantum information. These chips harness the principles of quantum mechanics to execute computations at speeds that were once unimaginable with classical computers. The significance of quantum chips in quantum computing cannot be emphasized enough, as they serve as the foundational elements driving progress in this state-of-the-art technology.

## Latest Trends in Quantum Chips and Computing

The progress in quantum chip technology has been rapidly accelerating, with major industry players investing heavily in research and development. Collaborations between leading companies like IBM, Google, and Intel have resulted in significant advancements in quantum chip design and fabrication. These developments are paving the way for the realization of practical quantum computers capable of addressing real-world challenges with unparalleled efficiency.

## Key Players in Quantum Chips and Computing

IBM, Google, and Intel stand at the forefront of quantum chips and computing, spearheading innovation and research efforts in the field. IBM's quantum computing research division is committed to pushing the boundaries of quantum technology, while Google's Quantum AI Lab is dedicated to the development of quantum algorithms and applications. Intel, a dominant force in semiconductor technology, is also venturing into quantum computing to maintain its competitiveness in the ever-evolving tech sector.

## Noteworthy News in Quantum Chips and Computing

Recent breakthroughs in quantum chip development have been garnering attention, highlighting the remarkable progress achieved in the field. The impact of quantum computing on diverse industries, ranging from finance to healthcare, is being explored as businesses recognize the potential for quantum technology to revolutionize their operations. With quantum chips becoming increasingly powerful and scalable, the opportunities for quantum computing are boundless.

## Target Audience Analysis

Our content on quantum chips and computing is tailored for tech enthusiasts intrigued by cutting-edge technology, IT professionals seeking to stay abreast of the latest advancements in quantum computing, and business leaders interested in understanding the potential applications of quantum technology. By offering valuable insights and updates on the subject, we aim to engage and educate our readers on the dynamic realm of quantum technology.

## Call to Action

As we delve deeper into the realm of quantum chips and computing, I urge readers to explore further into this transformative technology and its implications for the future. For additional resources, consider exploring IBM Research's "Quantum Computing and Quantum Chips," Google Quantum AI Lab's "Quantum Computing Explained," and Intel Newsroom's "The Future of Quantum Computing." Immerse yourself in the world of quantum technology and unlock the potential of quantum computing with us.

In conclusion, quantum chips are revolutionizing the computing landscape, unlocking new possibilities for solving intricate problems and catalyzing innovation across various industries. Stay tuned for more updates and insights on quantum chips and computing as we continue to unravel the boundless potential of quantum technology.